In [1]:
import pandas as pd
import os
pd.set_option('display.max_columns', None)

# Definimos las rutas
ruta_historial = '../data/raw/New_HistConsultas.csv'
ruta_usuarios = '../data/raw/New_Usuarios.csv'

In [2]:
# Carga de archivos.
try:
    # Encoding='latin-1' para que lea los caracteres en español
    # sep=';' 
    df_consultas_raw = pd.read_csv(ruta_historial, encoding='latin-1', sep=';') 
    df_usuarios_raw = pd.read_csv(ruta_usuarios, encoding='latin-1', sep=';')
    print("Las bases cargaron bien.")
except Exception as e:
    print("Hubo un error cargando los archivos")
    print(e)

# Revision rapida para ver que columnas trajo y si cruzan bien
if 'df_consultas_raw' in locals() and 'df_usuarios_raw' in locals():
    print("Dimensiones Historial:", df_consultas_raw.shape)
    print("Dimensiones Usuarios:", df_usuarios_raw.shape)

    print("Columnas Historial:", df_consultas_raw.columns.tolist())
    print("Columnas Usuarios:", df_usuarios_raw.columns.tolist())

Las bases cargaron bien.
Dimensiones Historial: (1282960, 4)
Dimensiones Usuarios: (337051, 22)
Columnas Historial: ['Unnamed: 0', 'ID_Cuenta', 'Fecha_consulta', 'Tipo_consulta']
Columnas Usuarios: ['ID_Cuenta', 'Tipo_persona', 'Departamento', 'Tiene_plan_avanzado', 'Tipo_Plan', 'Es_moroso', 'Tiene_plus', 'Ha_caido_mora', 'Correo', 'usa_app', 'Forma_pago', 'Monto_adeudado', 'Edad', 'Estrato', 'Motivo_llamada', 'Duracion_llamada', 'Tiempo_en_espera', 'Transferencia_llamada', 'primera_llamada', 'Antiguedad', 'Recomienda_marca', 'y']


In [3]:
# Muestro las primeras filas para validar el contenido
display(df_consultas_raw.head())

,Unnamed: 0,ID_Cuenta,Fecha_consulta,Tipo_consulta
0,0,340,2015-06-22 16:14:35,Actualización datos
1,1,340,2015-06-22 16:37:02,Agendamiento citas
2,2,340,2015-06-22 15:50:56,Agendamiento citas
3,3,4d5c0e16-9bef-8334-687e-55f043e02c62,2015-09-09 14:35:37,Agendamiento citas
4,4,66df1d0e-0d2c-1e44-aa3a-55b2a013640c,2015-07-24 20:36:26,Agendamiento citas


In [4]:
# Muestro las primeras filas para validar el contenido
display(df_usuarios_raw.head())

,ID_Cuenta,Tipo_persona,Departamento,Tiene_plan_avanzado,Tipo_Plan,Es_moroso,Tiene_plus,Ha_caido_mora,Correo,usa_app,Forma_pago,Monto_adeudado,Edad,Estrato,Motivo_llamada,Duracion_llamada,Tiempo_en_espera,Transferencia_llamada,primera_llamada,Antiguedad,Recomienda_marca,y
0,340,soltero,Santafé de Bogotá,si,f,no,no,no,no,si,mensajero,16475.149938,36,NaN,m21,114.522845,15.663706,no,no,mid-age,no,0
1,4d5c0e16-9bef-8334-687e-55f043e02c62,soltero,Santafé de Bogotá,no,f,no,si,si,si,no,mensajero,30167.091798,65,3,m17,124.262540,55.257504,no,no,new-new,si,0
2,66df1d0e-0d2c-1e44-aa3a-55b2a013640c,soltero,Santafé de Bogotá,no,f,no,si,no,no,si,tienda,11842.044402,30,2,m10,178.120444,4.635882,no,no,mid-age,si,0
3,1640de7c-ba08-cdfc-c21b-517fd7c5a259,soltero,Santafé de Bogotá,no,f,si,si,no,no,si,online,39601.288181,32,4,m18,332.970456,30.698536,no,no,Legend,no,0
4,e820f090-f4e7-eb8b-677c-55808c868b6f,unión libre,Santafé de Bogotá,no,b,no,si,si,si,si,online,36126.943210,70,3,m2,70.864127,44.999640,si,si,Legend,no,0


In [5]:
# Vamos a agrupar por ID para ver cuantos usuarios DIFERENTES tienen el mismo ID.
# Si el ID es unico, el conteo deberia ser 1.

conteo_duplicados = df_usuarios_raw['ID_Cuenta'].value_counts()

# Filtramos solo los que aparecen mas de una vez
ids_problematicos = conteo_duplicados[conteo_duplicados > 1]

print(f"Total de IDs que estan repetidos en la base de Usuarios: {len(ids_problematicos)}")
print("Top 10 de IDs mas repetidos (posibles IDs genericos o basura):")
print(ids_problematicos.head(10))

Total de IDs que estan repetidos en la base de Usuarios: 16878
Top 10 de IDs mas repetidos (posibles IDs genericos o basura):
ID_Cuenta
121314        14
1234          13
1022390282     8
1              7
30881871       6
ANONIMO1       5
12345          4
23621188       4
900475036      4
93436751       4
Name: count, dtype: int64


In [6]:
# Tomemos el caso mas critico (el ID '1' o '1234') para ver sus diferencias
id_ejemplo = '121314'  # O usa '1234' si prefieres

print(f"Perfiles encontrados para el ID '{id_ejemplo}':")

# Seleccionamos columnas clave para ver que son personas distintas
cols_clave = ['ID_Cuenta', 'Departamento', 'Edad', 'Estado_Civil', 'Tipo_Plan'] 
# Ajusta 'Estado_Civil' si la columna se llama diferente en tu df real

subset_problematico = df_usuarios_raw[df_usuarios_raw['ID_Cuenta'] == id_ejemplo]
display(subset_problematico)

print("Conclusion: El mismo ID tiene edades y ubicaciones diferentes. No se puede cruzar.")

Perfiles encontrados para el ID '121314':


,ID_Cuenta,Tipo_persona,Departamento,Tiene_plan_avanzado,Tipo_Plan,Es_moroso,Tiene_plus,Ha_caido_mora,Correo,usa_app,Forma_pago,Monto_adeudado,Edad,Estrato,Motivo_llamada,Duracion_llamada,Tiempo_en_espera,Transferencia_llamada,primera_llamada,Antiguedad,Recomienda_marca,y
104,121314,/casado.,Santafé de Bogotá,si,NaN,no,si,si,si,no,online,1810.971026,33,3,m10,6.142296,26.494558,no,no,Legend,no,1
105,121314,soltero-casado.,Santander,no,b,no,si,si,si,si,mensajero,14842.198745,41,2,m6,61.179562,12.539900,no,si,Legend,si,0
106,121314,soltero,Atlántico,no,d,no,no,no,si,si,NaN,10497.808836,46,2-3,m24,146.065168,1.969767,no,no,Legend,no,1
107,121314,soltero,Cundinamarca,no,00000,no,no,no,si,si,NaN,26683.898075,29,NaN,m15,244.117041,9.651768,si,no,Legend,no,0
108,121314,soltero,Nariño,no,b,no,si,si,si,no,online,38170.793074,70,2-3,m27,218.972499,10.772624,no,si,Young,no,1
109,121314,soltero,Exterior,no,a,no,si,no,si,no,mensajero,21612.449686,45,2-3,m24,151.388142,16.621477,no,no,new-new,si,1
110,121314,soltero,Norte de Santander,no,c,no,si,no,no,no,online,13785.787004,30,2-3,m33,66.652882,12.557435,no,no,Legend,si,1
111,121314,soltero,Arauca,no,00000,no,si,si,si,si,online,5847.460595,23,6,m29,48.268603,7.698572,no,si,mid-age,si,0
112,121314,soltero,Valle del Cauca,no,NaN,no,no,si,no,no,online,2978.983098,30,NaN,m33,48.333182,11.115522,no,no,new-new,si,0
113,121314,soltero,Boyacá,no,f,no,si,no,si,no,online,23883.150749,39,6,m14,198.401914,37.800036,no,no,new-new,si,1


Conclusion: El mismo ID tiene edades y ubicaciones diferentes. No se puede cruzar.


In [7]:
# Paso 1: Identificar IDs que son unicos de verdad
ids_unicos_reales = conteo_duplicados[conteo_duplicados == 1].index

# Paso 2: Filtrar la base de usuarios para dejar solo los que tienen IDs confiables
df_usuarios_clean = df_usuarios_raw[df_usuarios_raw['ID_Cuenta'].isin(ids_unicos_reales)].copy()

print(f"Usuarios originales: {df_usuarios_raw.shape[0]}")
print(f"Usuarios con IDs confiables: {df_usuarios_clean.shape[0]}")
print(f"Registros ambiguos eliminados: {df_usuarios_raw.shape[0] - df_usuarios_clean.shape[0]}")

# Ahora si podriamos hacer el merge con df_usuarios_clean

Usuarios originales: 337051
Usuarios con IDs confiables: 302984
Registros ambiguos eliminados: 34067


In [8]:
# 1. Validación de Formato de IDs
# Revisamos si hay caracteres extraños o longitudes inconsistentes en ID_Cliente
# (Asumiendo que ID_Cliente es la llave, si es ID_Cuenta ajustamos la variable)
print("--- Análisis de IDs en Consultas ---")
print(f"Total registros: {len(df_consultas_raw)}")
print(f"IDs únicos de clientes que consultaron: {df_consultas_raw['ID_Cuenta'].nunique()}")

# Chequeo de IDs nulos
ids_nulos = df_consultas_raw['ID_Cuenta'].isnull().sum()
print(f"Registros sin ID de cliente: {ids_nulos}")

# Chequeo de IDs con formatos sospechosos (ej. longitudes diferentes a la moda)
# Esto detecta si hay IDs tipo 'R-234' mezclados con '234'
df_consultas_raw['len_id'] = df_consultas_raw['ID_Cuenta'].astype(str).apply(len)
print("\nDistribución de longitud de los IDs:")
print(df_consultas_raw['len_id'].value_counts())

# Eliminamos la columna auxiliar
df_consultas_raw.drop(columns=['len_id'], inplace=True)

--- Análisis de IDs en Consultas ---
Total registros: 1282960
IDs únicos de clientes que consultaron: 319862
Registros sin ID de cliente: 0

Distribución de longitud de los IDs:
len_id
8     691770
9     312226
10    209736
7      51810
6      11698
11      3008
5       1290
4        760
12       140
36       138
14        98
1         94
13        84
3         64
2         40
17         2
16         2
Name: count, dtype: int64


Al auditar la variable ID_Cliente en el historial de consultas, detecté una dispersión anómala en la longitud de los registros:

    Datos Basura: Existen registros con identificadores de 1 a 5 dígitos (ej. 94 casos con longitud 1) y otros extremos de 17 dígitos. Esto sugiere pruebas de sistema o errores de digitación en el canal de atención.

    Distribución Real: La gran masa de datos se concentra entre los 7 y 10 dígitos, lo cual es consistente con el formato de documentos de identidad nacionales.

In [9]:
# Cruzamos el historial con los usuarios limpios (df_usuarios_clean).
df_master = pd.merge(df_consultas_raw, df_usuarios_clean, on='ID_Cuenta', how='inner')

#Limpieza inmediata de columnas basura
if 'Unnamed: 0' in df_master.columns:
    df_master.drop(columns=['Unnamed: 0'], inplace=True)

# 4Validación de Volumetría
print(f"Dimensiones del Dataset Maestro: {df_master.shape}")
print(f"Registros descartados del historial: {len(df_consultas_raw) - len(df_master)}")

Dimensiones del Dataset Maestro: (1140532, 24)
Registros descartados del historial: 142428


In [10]:
# --- Auditoría de Calidad Post-Merge ---

# A. Revisión de Tipos de Datos
print("--- Tipos de Datos ---")
print(df_master.dtypes)

# B. Detección de Valores Nulos
# Importante ver si 'Motivo_llamada' o 'Forma_pago' (que vienen de Usuarios) tienen huecos
nulls = df_master.isnull().sum()
print("\n--- Variables con Valores Nulos ---")
print(nulls[nulls > 0])

# C. Estadísticas de Variables Numéricas
# Revisamos las variables de tiempo y montos que venían en la tabla de Usuarios
cols_num = ['Monto_adeudado', 'Duracion_llamada', 'Tiempo_en_espera', 'Edad', 'Antiguedad']
print("\n--- Estadísticas Descriptivas (Numéricas) ---")
# Filtramos solo las columnas que existen en el df final para evitar errores
cols_existentes = [c for c in cols_num if c in df_master.columns]
print(df_master[cols_existentes].describe().round(2))

# D. Revisión de Categorías Clave
cols_cat = ['Tipo_consulta', 'Motivo_llamada', 'Estrato', 'y'] # Incluimos 'y' para ver balanceo
print("\n--- Distribución de Categorías Clave ---")
for col in cols_cat:
    if col in df_master.columns:
        print(f"\nVariable: {col}")
        print(df_master[col].value_counts(dropna=False).head(5))

--- Tipos de Datos ---
ID_Cuenta                 object
Fecha_consulta            object
Tipo_consulta             object
Tipo_persona              object
Departamento              object
Tiene_plan_avanzado       object
Tipo_Plan                 object
Es_moroso                 object
Tiene_plus                object
Ha_caido_mora             object
Correo                    object
usa_app                   object
Forma_pago                object
Monto_adeudado           float64
Edad                       int64
Estrato                   object
Motivo_llamada            object
Duracion_llamada         float64
Tiempo_en_espera         float64
Transferencia_llamada     object
primera_llamada           object
Antiguedad                object
Recomienda_marca          object
y                          int64
dtype: object

--- Variables con Valores Nulos ---
Tipo_persona       22952
Tipo_Plan         112924
Forma_pago        285604
Estrato           112984
Motivo_llamada      1076
dtype: in

In [11]:
#TRATAMIENTO DE DATOS

# 1. Conversión de Fechas
df_master['Fecha_consulta'] = pd.to_datetime(df_master['Fecha_consulta'], errors='coerce')

In [12]:
# 2. Corrección de Variables Numéricas (El paso que nos faltó)
# Forzamos 'Edad' a ser numérica. Lo que sea texto (ej: "No_Registrado") se vuelve NaN
df_master['Edad'] = pd.to_numeric(df_master['Edad'], errors='coerce')

In [13]:
# 3. Imputación de Variables Categóricas (Solo columnas de texto)
# Definimos explícitamente cuáles son las categóricas para no ensuciar las numéricas
cols_categoricas_imputar = ['Tipo_persona', 'Tipo_Plan', 'Forma_pago', 'Estrato', 'Motivo_llamada']
# Solo rellenamos nulos en estas columnas
df_master[cols_categoricas_imputar] = df_master[cols_categoricas_imputar].fillna('No_Registrado')

In [14]:
# 4. Limpieza de Textos (Strip)
cols_texto = df_master.select_dtypes(include=['object']).columns
for col in cols_texto:
    df_master[col] = df_master[col].str.strip()

In [15]:
# 5. Validación Final de Tipos
print("--- Tipos de Datos Finales ---")
print(df_master.dtypes)

--- Tipos de Datos Finales ---
ID_Cuenta                        object
Fecha_consulta           datetime64[ns]
Tipo_consulta                    object
Tipo_persona                     object
Departamento                     object
Tiene_plan_avanzado              object
Tipo_Plan                        object
Es_moroso                        object
Tiene_plus                       object
Ha_caido_mora                    object
Correo                           object
usa_app                          object
Forma_pago                       object
Monto_adeudado                  float64
Edad                              int64
Estrato                          object
Motivo_llamada                   object
Duracion_llamada                float64
Tiempo_en_espera                float64
Transferencia_llamada            object
primera_llamada                  object
Antiguedad                       object
Recomienda_marca                 object
y                                 int64
dtype: ob

In [17]:

# 6. Guardado Definitivo
df_master.to_csv('../data/procesada/master_dataset.csv', index=False)
print("Corrección aplicada. 'master_dataset.csv' sobrescrito correctamente.")

Corrección aplicada. 'master_dataset.csv' sobrescrito correctamente.
